In [1]:
from collections import Counter
import pandas as pd
import numpy as np
import xgboost as xgb
pd.set_option('display.max_colwidth', -1)

In [2]:
# for autoreload modules
%load_ext autoreload
%autoreload 2

In [3]:
from utils import UtilsKy

In [77]:
# kyw3
# db_teach = pd.read_csv(UtilsKy.DB_TEACH_KYW3, dtype=str)
# db_test = pd.read_csv(UtilsKy.DB_TEST_KYW3, dtype=str)
# white = pd.read_csv(UtilsKy.WHITE_KYW3 , dtype=str)

In [12]:
# ky9
db_teach = pd.read_csv(UtilsKy.DB_TEACH_KY9, dtype=str)
db_test = pd.read_csv(UtilsKy.DB_TEST_KY9, dtype=str)
white = pd.read_csv(UtilsKy.WHITE_KY9 , dtype=str)

In [5]:
COL_FACTORS = [ 'amount', 'bank_currency', 'hour', 'day_of_week', 
               'bin', 'longitude', 'latitude', 'phone_2_norm']

In [6]:
from analyzer import HelperAnalyzer, AnalyzerPrediction

In [7]:
train = db_teach[COL_FACTORS]
test = db_test[COL_FACTORS]

In [8]:
train = train.apply(pd.to_numeric, errors="coerce")
test = test.apply(pd.to_numeric, errors="coerce")

In [9]:
replace_val = -9999
train.fillna(replace_val)
test.fillna(replace_val)
replace_val

-9999

In [10]:
train = train.values
test = test.values
label = db_teach.status

In [13]:
analyzer_prediction =  AnalyzerPrediction(db_teach, db_test, white)

In [21]:
result_df_amount = None
weight = analyzer_prediction.get_xgb_weight()

In [36]:
# 1 best
# max_depths =[5]
# nrounds = [90]
# etas = [0.2]

# 2
# max_depths =[3]
# nrounds = [120]
# etas = [0.1]

# 3
# max_depths =[5]
# nrounds = [80]
# etas = [0.2, 0.25, 0.3]

# 3
max_depths =[3]
nrounds = [80]
etas = [0.3, 0.35, 0.4]



In [37]:
for nround in nrounds:
    for eta in etas:
        for max_depth in max_depths:
            config = {'max_depth':max_depth, 'learning_rate':eta
                      ,'n_estimators':nround }
            model = xgb.XGBClassifier(**config)
            
            model.fit(train, label, eval_metric = 'auc', sample_weight=weight)
             
            test_pred = model.predict_proba(test)
            db_test["probability"] = test_pred[:, 1]
            description = str(config) + 'wl'
            description = '-' . join([str(elem) for elem in (max_depth, eta, nround)])           
            result_df_amount = analyzer_prediction.get_table_prediction(description=description, result_df=result_df_amount, metric="amount")

In [32]:
result_df_amount

,description,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_10,p_20,...,n_test_in_wl,n_test_bad_in_wl,amount_test_in_wl,amount_test_bad_in_wl,n_teach,n_teach_bad,n_test,n_test_bad,amount_test_bad,amount_test
0,5-0.2-90,4.450000,6.520000,8.490000,10.610000,13.140000,15.690000,17.930000,26.600000,47.850000,...,289796,294714,2.025618e+07,2.083261e+07,742167,42005,603378,5642,660225.66,4.549701e+07
1,threshold amount,0.760708,0.723037,0.703265,0.687484,0.674667,0.662930,0.653549,0.629152,0.558903,...,289796,294714,2.025618e+07,2.083261e+07,742167,42005,603378,5642,660225.66,4.549701e+07
2,3-0.1-120,2.100000,4.480000,6.430000,8.850000,11.340000,14.260000,15.850000,26.310000,47.410000,...,289796,294714,2.025618e+07,2.083261e+07,742167,42005,603378,5642,660225.66,4.549701e+07
3,threshold amount,0.717123,0.696156,0.682263,0.671375,0.663133,0.654717,0.648278,0.631389,0.577599,...,289796,294714,2.025618e+07,2.083261e+07,742167,42005,603378,5642,660225.66,4.549701e+07
4,5-0.2-80,4.530000,6.490000,8.670000,11.080000,13.860000,16.730000,18.640000,26.600000,47.410000,...,289796,294714,2.025618e+07,2.083261e+07,742167,42005,603378,5642,660225.66,4.549701e+07
5,threshold amount,0.757860,0.722540,0.702001,0.686233,0.673828,0.662838,0.653437,0.629832,0.560425,...,289796,294714,2.025618e+07,2.083261e+07,742167,42005,603378,5642,660225.66,4.549701e+07
6,5-0.25-80,4.670000,7.530000,9.610000,11.710000,14.330000,15.780000,17.890000,23.780000,48.650000,...,289796,294714,2.025618e+07,2.083261e+07,742167,42005,603378,5642,660225.66,4.549701e+07
7,threshold amount,0.756475,0.724744,0.704510,0.689012,0.677242,0.667214,0.657959,0.633386,0.557548,...,289796,294714,2.025618e+07,2.083261e+07,742167,42005,603378,5642,660225.66,4.549701e+07
8,5-0.3-80,4.200000,6.970000,9.390000,11.680000,14.910000,16.650000,18.000000,24.930000,47.570000,...,289796,294714,2.025618e+07,2.083261e+07,742167,42005,603378,5642,660225.66,4.549701e+07
9,threshold amount,0.764420,0.732041,0.711664,0.695627,0.682918,0.670778,0.660447,0.633627,0.554977,...,289796,294714,2.025618e+07,2.083261e+07,742167,42005,603378,5642,660225.66,4.549701e+07


In [33]:
n = result_df_amount.shape[0]
sub_rows = list(range(n))[::2]
stat_best = result_df_amount.copy().iloc[sub_rows,:]

col_names = [col for col in stat_best.columns if col.startswith('p_') ]
stat_best.loc[:, col_names] = stat_best.loc[:, col_names].astype(float)

stat_best = stat_best.sort_values(by="rating", ascending=False)
stat_best = stat_best.sort_values(by="p_5", ascending=False)

In [35]:
stat_best.iloc[:,0:11]

,description,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_10,p_20,rating
8,5-0.3-80,4.20,6.97,9.39,11.68,14.91,16.65,18.00,24.93,47.57,47.15
6,5-0.25-80,4.67,7.53,9.61,11.71,14.33,15.78,17.89,23.78,48.65,47.85
4,5-0.2-80,4.53,6.49,8.67,11.08,13.86,16.73,18.64,26.60,47.41,44.63
0,5-0.2-90,4.45,6.52,8.49,10.61,13.14,15.69,17.93,26.60,47.85,43.21
2,3-0.1-120,2.10,4.48,6.43,8.85,11.34,14.26,15.85,26.31,47.41,33.20


,description,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_10,p_20,...,n_test_in_wl,n_test_bad_in_wl,amount_test_in_wl,amount_test_bad_in_wl,n_teach,n_teach_bad,n_test,n_test_bad,amount_test_bad,amount_test
0,5-0.2-90,4.450000,6.520000,8.490000,10.610000,13.140000,15.69000,17.930000,26.600000,47.850000,...,289796,294714,2.025618e+07,2.083261e+07,742167,42005,603378,5642,660225.66,4.549701e+07
1,threshold amount,0.760708,0.723037,0.703265,0.687484,0.674667,0.66293,0.653549,0.629152,0.558903,...,289796,294714,2.025618e+07,2.083261e+07,742167,42005,603378,5642,660225.66,4.549701e+07


In [103]:
#stat_best

,description,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_10,p_20,...,n_test_in_wl,n_test_bad_in_wl,amount_test_in_wl,amount_test_bad_in_wl,n_teach,n_teach_bad,n_test,n_test_bad,amount_test_bad,amount_test
0,,5.03,8.7,12.39,15.69,18.86,21.53,24.49,31.76,52.3,...,289796,294714,2.025618e+07,2.083261e+07,742167,42005,603378,5642,660225.66,4.549701e+07
